In [36]:
import numpy as np
import cv2

In [38]:
import cv2
image_file = "temp/print_noslen.png"
img =  cv2.imread(image_file)

In [39]:
cv2.startWindowThread()
cv2.imshow("original image", img)
cv2.waitKey(5000)
cv2.destroyAllWindows()
cv2.waitKey(10)


-1

Inverted Image

In [5]:
inverted_image = cv2.bitwise_not(img)
cv2.imwrite("temp/inverted.jpg", inverted_image)

True

In [6]:
img = cv2.imread("temp/inverted.jpg")
cv2.imshow("inverted", img)
cv2.waitKey(5000)
cv2.destroyAllWindows()
cv2.waitKey(10)

-1

# 02: Rescaling

# 03: Binarization

In [40]:
def grayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)


In [41]:
img = cv2.imread("temp/print_noslen.png")
gray_image = grayscale(img)
cv2.imwrite("temp/noslen_gray.jpg", gray_image)

True

In [42]:
img = cv2.imread("temp/noslen_gray.jpg")
cv2.imshow("gray", img)
cv2.waitKey(5000)
cv2.destroyAllWindows()
cv2.waitKey(10)

-1

In [44]:
thresh, im_bw = cv2.threshold(gray_image, 210, 230, cv2.THRESH_BINARY)
cv2.imwrite("temp/noslen_gray.jpg", im_bw)

True

In [45]:
img = cv2.imread("temp/noslen_gray.jpg")
cv2.imshow("gray", img)
cv2.waitKey(5000)
cv2.destroyAllWindows()
cv2.waitKey(10)

-1

# Noise Removal

In [47]:
def noise_removal(image):
    kernel = np.ones((1,1), np.uint8)
    image = cv2.dilate(image, kernel, iterations= 1)
    kernel = np.ones((1,1), np.uint8)
    image = cv2.erode(image, kernel, iterations= 1)
    image = cv2.morphologyEx(image, cv2.MORPH_CLOSE, kernel)
    image = cv2.medianBlur(image, 3)
    return(image)

In [48]:
no_noise = noise_removal(im_bw)
cv2.imwrite("temp/noslen_no_noise.jpg", no_noise)

True

In [49]:
img = cv2.imread("temp/noslen_no_noise.jpg")
cv2.imshow("gray", img)
cv2.waitKey(5000)
cv2.destroyAllWindows()
cv2.waitKey(10)

-1

# Erosão

In [15]:
def thin_font(image):
    image = cv2.bitwise_not(image)
    kernel = np.ones((2,2), np.uint8)
    image = cv2.erode(image, kernel, iterations = 1)
    image = cv2.bitwise_not(image)
    return(image)

In [16]:
eroded_image = thin_font(no_noise)
cv2.imwrite("temp/erodede_image.jpg", eroded_image)

True

In [17]:
img = cv2.imread("temp/erodede_image.jpg")
cv2.imshow("gray", img)
cv2.waitKey(5000)
cv2.destroyAllWindows()
cv2.waitKey(10)

-1

# Dilatação

In [18]:
def thick_font(image):
    image = cv2.bitwise_not(image)
    kernel = np.ones((2,2), np.uint8)
    image = cv2.dilate(image, kernel, iterations=1)
    image = cv2.bitwise_not(image)
    return(image)


In [19]:
dilated_image = thick_font(no_noise)
cv2.imwrite("temp/dilated_image.jpg", dilated_image)

True

In [20]:
img = cv2.imread("temp/dilated_image.jpg")
cv2.imshow("gray", img)
cv2.waitKey(5000)
cv2.destroyAllWindows()
cv2.waitKey(10)

-1

# Reskewing

In [21]:
new = cv2.imread("temp/page_01_rotated.JPG")
cv2.imshow("temp/page_01_rotated.JPG", img)
cv2.waitKey(5000)
cv2.destroyAllWindows()
cv2.waitKey(10)

-1

In [22]:
#https://becominghuman.ai/how-to-automatically-deskew-straighten-a-text-image-using-opencv-a0c30aed83df
import numpy as np

def getSkewAngle(cvImage) -> float:
    # Prep image, copy, convert to gray scale, blur, and threshold
    newImage = cvImage.copy()
    gray = cv2.cvtColor(newImage, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (9, 9), 0)
    thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

    # Apply dilate to merge text into meaningful lines/paragraphs.
    # Use larger kernel on X axis to merge characters into single line, cancelling out any spaces.
    # But use smaller kernel on Y axis to separate between different blocks of text
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (30, 5))
    dilate = cv2.dilate(thresh, kernel, iterations=2)

    # Find all contours
    contours, hierarchy = cv2.findContours(dilate, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key = cv2.contourArea, reverse = True)
    for c in contours:
        rect = cv2.boundingRect(c)
        x,y,w,h = rect
        cv2.rectangle(newImage,(x,y),(x+w,y+h),(0,255,0),2)

    # Find largest contour and surround in min area box
    largestContour = contours[0]
    print (len(contours))
    minAreaRect = cv2.minAreaRect(largestContour)
    cv2.imwrite("temp/boxes.jpg", newImage)
    # Determine the angle. Convert it to the value that was originally used to obtain skewed image
    angle = minAreaRect[-1]
    if angle < -45:
        angle = 90 + angle
    return -1.0 * angle

# Rotate the image around its center
def rotateImage(cvImage, angle: float):
    newImage = cvImage.copy()
    (h, w) = newImage.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    newImage = cv2.warpAffine(newImage, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
    return newImage

In [23]:
def deskew(img):
    angle = getSkewAngle(img)
    return(rotateImage(img, -1.0 * angle))

In [25]:
straight_image = deskew(new)
cv2.imwrite("temp/fix_rotated_img.jpg", straight_image)

28


True

In [26]:
fixed = cv2.imread("temp/fix_rotated_img.JPG")
cv2.imshow("temp/fix_rotated_img.JPG", fixed)
cv2.waitKey(5000)
cv2.destroyAllWindows()
cv2.waitKey(10)

-1

# Destroy Borders

In [27]:
fixed = cv2.imread("temp/no_noise.JPG")
cv2.imshow("temp/no_noise.JPG", fixed)
cv2.waitKey(5000)
cv2.destroyAllWindows()
cv2.waitKey(10)

-1

In [28]:
def remove_borders(image):
    contours, heiarchy = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cntsSorted = sorted(contours, key=lambda x:cv2.contourArea(x))
    cnt = cntsSorted[-1]
    x, y, w, h = cv2.boundingRect(cnt)
    crop = image[y:y+h, x:x+w]
    return (crop)

In [29]:
no_borders = remove_borders(no_noise)
cv2.imwrite("temp/no_borders.jpg", no_borders)

True

In [30]:
fixed = cv2.imread("temp/no_borders.JPG")
cv2.imshow("temp/no_borders.JPG", fixed)
cv2.waitKey(5000)
cv2.destroyAllWindows()
cv2.waitKey(10)

-1

# Create Borders

In [31]:
color = [255, 255, 255]
top, bottom, left, right = [150]*4

In [32]:
image_with_border = cv2.copyMakeBorder(no_borders, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)
cv2.imwrite("temp/image_with_border.jpg", image_with_border)
display("temp/image_with_border.jpg")

'temp/image_with_border.jpg'

In [35]:
fixed = cv2.imread("temp/image_with_border.JPG")
cv2.imshow("temp/image_with_border.JPG", fixed)
cv2.waitKey(5000)
cv2.destroyAllWindows()
cv2.waitKey(10)

-1